In [27]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import re, os

import pandas as pd

In [98]:
root_path = os.getcwd()

In [99]:
dset_path = os.path.join(root_path, "LFW_Dataset/")

## Results from CycleGAN
cycle_gan_folder = "pytorch-CycleGAN-and-pix2pix"
dataset = "LFW Dataset"
model_trained = "mask2unmask_final2"

trainA_path = os.path.join(root_path, cycle_gan_folder, "datasets", "LFW_dataset", "trainA")
trainB_path = os.path.join(root_path, cycle_gan_folder, "datasets", "LFW_dataset", "trainB")

testA_path = os.path.join(root_path, cycle_gan_folder, "datasets", "LFW_dataset", "testA")
testB_path = os.path.join(root_path, cycle_gan_folder, "datasets", "LFW_dataset", "testB")

In [102]:
def extract_files(folder_path):
    paths = []
    for root, dirs, files in os.walk(folder_path, topdown=False):
        for name in files:
            paths.append((root, name))
    return paths
    
def create_df_from_real(paths):
    df = pd.DataFrame(paths, columns=['root', 'filename'])
    
    df['fullpath'] = df['root'] + '/' + df['filename']
    df = df.drop(columns="root")
    df['person'] = df['filename'].apply(lambda x: " ".join(x.split("_")[:-1]))
    
    df["name"] = df["filename"].apply(lambda x: "_".join(x.split("_")[:-1]))
    
    return df

In [103]:
#Getting results from CycleGAN

trainA_urls = extract_files(trainA_path)
trainB_urls = extract_files(trainA_path)

testA_urls = extract_files(testA_path)
testB_urls = extract_files(testB_path)

In [104]:
trainA  = create_df_from_real(trainA_urls)
trainB  = create_df_from_real(trainB_urls)

testA  = create_df_from_real(testA_urls)
testB  = create_df_from_real(testB_urls)

In [106]:
trainB["person"]

0          Scott McClellan
1             Alvaro Uribe
2            Roger Federer
3             Bertie Ahern
4            Joe Lieberman
               ...        
1264            JK Rowling
1265          Roh Moo-hyun
1266       Renee Zellweger
1267    Rubens Barrichello
1268         Tung Chee-hwa
Name: person, Length: 1269, dtype: object

In [107]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [110]:
from torch.utils.data import Dataset
from torchvision.io import read_image


data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224, scale=(.5, 1.)),
        transforms.AutoAugment(transforms.AutoAugmentPolicy.IMAGENET),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

class PandasDataset(Dataset):
    def __init__(self, df: pd.DataFrame, transform=None, target_transform=None,
                 path_col = 'fullpath', label_col = 'person'):
        self.df = df.copy()
        self.img_paths = df[path_col]
        self.labels = df[label_col]
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        path = self.img_paths.iloc[idx]
        image = Image.open(path)
        label = self.labels.iloc[idx]


        if self.transform:
            image = self.transform(image)

        if self.target_transform:
            label = self.target_transform(label)

        return image, label

def tokenize_names(name: str):
    return name_to_idx[name]

In [112]:
import json

current_path = os.getcwd()
tokenz_path = os.path.join(current_path, "tokens.json")
f = open(tokenz_path)
name_to_idx = json.load(f)

In [113]:
name_to_idx

{'Abdullah Gul': 0,
 'Adrien Brody': 1,
 'Ahmed Chalabi': 2,
 'Ai Sugiyama': 3,
 'Al Gore': 4,
 'Al Sharpton': 5,
 'Alan Greenspan': 6,
 'Alastair Campbell': 7,
 'Albert Costa': 8,
 'Alejandro Toledo': 9,
 'Ali Naimi': 10,
 'Allyson Felix': 11,
 'Alvaro Uribe': 12,
 'Amelia Vega': 13,
 'Amelie Mauresmo': 14,
 'Ana Guevara': 15,
 'Ana Palacio': 16,
 'Andre Agassi': 17,
 'Andy Roddick': 18,
 'Angela Bassett': 19,
 'Angela Merkel': 20,
 'Angelina Jolie': 21,
 'Ann Veneman': 22,
 'Anna Kournikova': 23,
 'Antonio Banderas': 24,
 'Antonio Palocci': 25,
 'Ari Fleischer': 26,
 'Ariel Sharon': 27,
 'Arminio Fraga': 28,
 'Arnold Schwarzenegger': 29,
 'Arnoldo Aleman': 30,
 'Ashanti': 31,
 'Atal Bihari Vajpayee': 32,
 'Ben Affleck': 33,
 'Benazir Bhutto': 34,
 'Benjamin Netanyahu': 35,
 'Bernard Law': 36,
 'Bertie Ahern': 37,
 'Bill Clinton': 38,
 'Bill Frist': 39,
 'Bill Gates': 40,
 'Bill Graham': 41,
 'Bill McBride': 42,
 'Bill Simon': 43,
 'Billy Crystal': 44,
 'Binyamin Ben-Eliezer': 45,
 'B

In [114]:
# train_naked, train_masked, test_naked, test_masked
# trainA, trainB, testA, testB respectively

## Naked Model

In [67]:
dset_trainA = PandasDataset(trainA, transform=data_transforms["train"], target_transform=tokenize_names)
dset_testA = PandasDataset(testA, transform=data_transforms["val"], target_transform=tokenize_names)

dl_trainA = torch.utils.data.DataLoader(dset_trainA, batch_size=32, shuffle=True, num_workers=4)
dl_testA = torch.utils.data.DataLoader(dset_testA, batch_size=32, shuffle=False, num_workers=4)

In [68]:
dataloaders = {"train": dl_trainA, "val": dl_testA}

In [71]:
import torchvision

model = torchvision.models.resnet18(pretrained=True)

if type(model) == torchvision.models.vgg.VGG:
  num_ftrs = model.classifier[6].in_features
  model.classifier[6] = nn.Linear(num_ftrs, len(name_to_idx))
else:
  num_ftrs = model.fc.in_features
  model.fc = nn.Linear(num_ftrs, len(name_to_idx))

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/imad/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [73]:
DEVICE = "cuda"
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [257]:
import time, copy, os

def train_model(model, criterion, dataloaders, optimizer, scheduler, num_epochs=25, cuda_id = 0):
    since = time.time()

    val_acc_history = []
    logs = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.cuda(cuda_id)
                labels = labels.cuda(cuda_id)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        scheduler.step

                # statistics
                running_loss += loss.item() * inputs.size(0)
                
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            logs.append('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, val_acc_history, logs

In [75]:
### Training

model = train_model(model, criterion, dataloaders, optimizer, exp_lr_scheduler, num_epochs = 50)

Epoch 0/49
----------
train Loss: 6.2482 Acc: 0.0016
val Loss: 6.1762 Acc: 0.0000

Epoch 1/49
----------
train Loss: 6.1465 Acc: 0.0016
val Loss: 6.1002 Acc: 0.0059

Epoch 2/49
----------
train Loss: 6.0437 Acc: 0.0024
val Loss: 6.0365 Acc: 0.0059

Epoch 3/49
----------
train Loss: 5.9599 Acc: 0.0118
val Loss: 5.9730 Acc: 0.0106

Epoch 4/49
----------
train Loss: 5.8804 Acc: 0.0189
val Loss: 5.9236 Acc: 0.0142

Epoch 5/49
----------
train Loss: 5.8050 Acc: 0.0205
val Loss: 5.8576 Acc: 0.0189

Epoch 6/49
----------
train Loss: 5.7091 Acc: 0.0433
val Loss: 5.7669 Acc: 0.0331

Epoch 7/49
----------
train Loss: 5.6048 Acc: 0.0709
val Loss: 5.6938 Acc: 0.0402

Epoch 8/49
----------
train Loss: 5.5098 Acc: 0.0867
val Loss: 5.6174 Acc: 0.0473

Epoch 9/49
----------
train Loss: 5.4074 Acc: 0.1017
val Loss: 5.5182 Acc: 0.0496

Epoch 10/49
----------
train Loss: 5.3013 Acc: 0.1560
val Loss: 5.4293 Acc: 0.0709

Epoch 11/49
----------
train Loss: 5.2001 Acc: 0.1600
val Loss: 5.3340 Acc: 0.0863

Ep

In [78]:
torch.save(model, 'pretrained/imad_resnet_naked.pt')

#### Evaluate on Fake Data

In [79]:
## Naked

type_model = "imad_resnet_"
### Load Model
dir_pretrained = os.path.join(root_path, "pretrained")
model_name = type_model + "naked.pt"
model_path = os.path.join(dir_pretrained, model_name)

model, scores, history = torch.load(model_path)

In [80]:
## Results from CycleGAN
cycle_gan_folder = "pytorch-CycleGAN-and-pix2pix"
dataset = "LFW Dataset"
model_trained = "mask2unmask_final2"

fake_naked_path = os.path.join(root_path, cycle_gan_folder, "results", model_trained, "fake_naked")

In [81]:
def extract_files(folder_path):
    paths = []
    for root, dirs, files in os.walk(folder_path, topdown=False):
        for name in files:
            paths.append((root, name))
    return paths
    
def create_df(paths):
    df = pd.DataFrame(paths, columns=['root', 'filename'])
    
    df['fullpath'] = df['root'] + '/' + df['filename']
    df = df.drop(columns="root")
    df["fake"]   = False
    df['person'] = df['filename'].apply(lambda x: " ".join(x.split("_")[:-2]))
    
    df['status'] = df['filename'].apply(lambda x: x.split("_")[-1][:-4])
    df["name"] = df["filename"].apply(lambda x: "_".join(x.split("_")[:-1]))
    df['fake'].loc[df['status'] == 'fake'] = True
    df = df.drop(columns="status")
    
    fake_df = df[df['fake']]
    fake_df = fake_df.drop(columns="fake")
    
    real_df = df[~df['fake']]
    real_df = real_df.drop(columns="fake")
    
    return real_df, fake_df

In [82]:
#Getting results from CycleGAN

fake_naked = extract_files(fake_naked_path)

In [83]:
fake_naked_real, fake_naked_fake = create_df(fake_naked)

/home/imad/anaconda3/envs/inf664/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [85]:
#Naked Fake dataloader
ds_fake_naked_fake = PandasDataset(fake_naked_fake, transform=data_transforms["val"], target_transform=tokenize_names)
dl_fake_naked_fake = torch.utils.data.DataLoader(ds_fake_naked_fake, batch_size=32, shuffle=False, num_workers=4)

In [87]:
dl_fake_naked_fake

In [89]:
from tqdm import tqdm

def test_accuracy(dl_, model):
    match = 0
    with torch.no_grad():
        for input, label in tqdm(dl_):
            input = input.cuda()
            out = model(input)
            pred_y = torch.argmax(out, dim=-1).cpu()
            match += torch.sum(pred_y == label)

    n_ = len(dl_.dataset)
    print('\n\tNumber of matches: {}, Number of samples: {}, Accuracy: {:.4f}'.format(match, n_, match/n_))

In [90]:
#Naked Fake
test_accuracy(dl_fake_naked_fake, model)

100%|██████████| 16/16 [00:01<00:00, 13.39it/s]


	Number of matches: 161, Number of samples: 500, Accuracy: 0.3220


----------------------------------------------------------------------------------------------------------

## Masked Model

In [115]:
import torchvision

model = torchvision.models.resnet18(pretrained=True)

if type(model) == torchvision.models.vgg.VGG:
  num_ftrs = model.classifier[6].in_features
  model.classifier[6] = nn.Linear(num_ftrs, len(name_to_idx))
else:
  num_ftrs = model.fc.in_features
  model.fc = nn.Linear(num_ftrs, len(name_to_idx))

In [120]:
len(trainB["person"].unique()) == len(testB["person"].unique())

True

In [123]:
dset_trainB = PandasDataset(trainB, transform=data_transforms["train"], target_transform=tokenize_names)
dset_testB = PandasDataset(testB, transform=data_transforms["val"], target_transform=tokenize_names)

dl_trainB = torch.utils.data.DataLoader(dset_trainB, batch_size=32, shuffle=True, num_workers=4)
dl_testB = torch.utils.data.DataLoader(dset_testB, batch_size=32, shuffle=False, num_workers=4)

In [124]:
dataloaders = {"train": dl_trainB, "val": dl_testB}

In [127]:
DEVICE = "cuda"
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [128]:
### Training
model = train_model(model, criterion, dataloaders, optimizer, exp_lr_scheduler, num_epochs = 50)

Epoch 0/49
----------
train Loss: 6.2623 Acc: 0.0008
val Loss: 6.2234 Acc: 0.0013

Epoch 1/49
----------
train Loss: 6.1582 Acc: 0.0024
val Loss: 6.1329 Acc: 0.0037

Epoch 2/49
----------
train Loss: 6.0415 Acc: 0.0032
val Loss: 6.0925 Acc: 0.0037

Epoch 3/49
----------
train Loss: 5.9560 Acc: 0.0071
val Loss: 6.0409 Acc: 0.0050

Epoch 4/49
----------
train Loss: 5.8963 Acc: 0.0102
val Loss: 5.9984 Acc: 0.0075

Epoch 5/49
----------
train Loss: 5.8163 Acc: 0.0252
val Loss: 5.9448 Acc: 0.0100

Epoch 6/49
----------
train Loss: 5.7308 Acc: 0.0370
val Loss: 5.8882 Acc: 0.0112

Epoch 7/49
----------
train Loss: 5.6294 Acc: 0.0575
val Loss: 5.8395 Acc: 0.0163

Epoch 8/49
----------
train Loss: 5.5603 Acc: 0.0709
val Loss: 5.7775 Acc: 0.0213

Epoch 9/49
----------
train Loss: 5.4290 Acc: 0.0930
val Loss: 5.7111 Acc: 0.0200

Epoch 10/49
----------
train Loss: 5.3304 Acc: 0.1221
val Loss: 5.6550 Acc: 0.0300

Epoch 11/49
----------
train Loss: 5.2423 Acc: 0.1647
val Loss: 5.5774 Acc: 0.0312

Ep

In [129]:
torch.save(model, 'pretrained/imad_resnet_masked.pt')

In [225]:
## Masked

type_model = "imad_resnet_"
### Load Model
dir_pretrained = os.path.join(root_path, "pretrained")
model_name = type_model + "masked.pt"
model_path = os.path.join(dir_pretrained, model_name)

model, scores, history = torch.load(model_path)

In [226]:
## Results from CycleGAN
cycle_gan_folder = "pytorch-CycleGAN-and-pix2pix"
dataset = "LFW Dataset"
model_trained = "mask2unmask_final2"

fake_masked_path = os.path.join(root_path, cycle_gan_folder, "results", model_trained, "fake_masked")

In [227]:
#Getting results from CycleGAN

fake_masked = extract_files(fake_masked_path)

In [228]:
_, fake_masked_fake = create_df(fake_masked)

In [229]:
dset_fake_masked = PandasDataset(fake_masked_fake, transform=data_transforms["val"], target_transform=tokenize_names)
dl_fake_masked = torch.utils.data.DataLoader(dset_fake_masked, batch_size=32, shuffle=False, num_workers=4)

In [230]:
#Naked Masked
test_accuracy(dl_fake_masked, model)

100%|██████████| 16/16 [00:01<00:00, 14.89it/s]


	Number of matches: 103, Number of samples: 500, Accuracy: 0.2060


----------------------------------------------------------------

## General Model

In [231]:
## Naked

type_model = "imad_resnet_"
### Load Model
dir_pretrained = os.path.join(root_path, "pretrained")
model_name = type_model + "general.pt"
model_path = os.path.join(dir_pretrained, model_name)

model, scores, history = torch.load(model_path)

In [232]:
## Results from CycleGAN
cycle_gan_folder = "pytorch-CycleGAN-and-pix2pix"
dataset = "LFW Dataset"
model_trained = "mask2unmask_final2"

fake_masked_path = os.path.join(root_path, cycle_gan_folder, "results", model_trained, "fake_masked")
fake_naked_path = os.path.join(root_path, cycle_gan_folder, "results", model_trained, "fake_naked")

In [233]:
#Getting results from CycleGAN

fake_naked = extract_files(fake_naked_path)
fake_masked = extract_files(fake_masked_path)

In [234]:
_, fake_naked_fake = create_df(fake_naked)
_, fake_masked_fake = create_df(fake_masked)

In [235]:
df_fake_all = pd.concat([fake_naked_fake, fake_masked_fake])

In [236]:
dset_fake_all = PandasDataset(df_fake_all, transform=data_transforms["val"], target_transform=tokenize_names)
dl_fake_all = torch.utils.data.DataLoader(dset_fake_all, batch_size=32, shuffle=False, num_workers=4)

In [237]:
#General model
test_accuracy(dl_fake_all, model)

100%|██████████| 32/32 [00:01<00:00, 19.66it/s]


	Number of matches: 457, Number of samples: 1000, Accuracy: 0.4570


#### Real Data on each subset

In [244]:
#testA: Naked
dset_fake_naked = PandasDataset(testA, transform=data_transforms["val"], target_transform=tokenize_names)
dl_fake_naked = torch.utils.data.DataLoader(dset_fake_naked, batch_size=32, shuffle=False, num_workers=4)

In [245]:
#General model on Faked Nake
test_accuracy(dl_fake_naked, model)

100%|██████████| 27/27 [00:02<00:00, 13.12it/s]


	Number of matches: 423, Number of samples: 846, Accuracy: 0.5000


In [247]:
#testB: Masked
dset_fake_masked = PandasDataset(testB, transform=data_transforms["val"], target_transform=tokenize_names)
dl_fake_masked = torch.utils.data.DataLoader(dset_fake_masked, batch_size=32, shuffle=False, num_workers=4)

In [248]:
#General model on Faked Nake
test_accuracy(dl_fake_masked, model)

100%|██████████| 25/25 [00:02<00:00, 12.46it/s]



	Number of matches: 395, Number of samples: 800, Accuracy: 0.4938


------------------------------------------------------------------------------------------

### Masked VGG16

In [250]:
import torchvision

model = torchvision.models.vgg16(pretrained=True)

if type(model) == torchvision.models.vgg.VGG:
  num_ftrs = model.classifier[6].in_features
  model.classifier[6] = nn.Linear(num_ftrs, len(name_to_idx))
else:
  num_ftrs = model.fc.in_features
  model.fc = nn.Linear(num_ftrs, len(name_to_idx))

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/imad/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [251]:
dset_trainB = PandasDataset(trainB, transform=data_transforms["train"], target_transform=tokenize_names)
dset_testB = PandasDataset(testB, transform=data_transforms["val"], target_transform=tokenize_names)

dl_trainB = torch.utils.data.DataLoader(dset_trainB, batch_size=32, shuffle=True, num_workers=4)
dl_testB = torch.utils.data.DataLoader(dset_testB, batch_size=32, shuffle=False, num_workers=4)

In [255]:
dataloaders = {"train": dl_trainB, "val": dl_testB}

DEVICE = "cuda"
model = model.cuda(1)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [258]:
### Training
model = train_model(model, criterion, dataloaders, optimizer, exp_lr_scheduler, num_epochs = 50, cuda_id=1)

Epoch 0/49
----------
train Loss: 6.1172 Acc: 0.0016
val Loss: 6.0530 Acc: 0.0037

Epoch 1/49
----------
train Loss: 6.0480 Acc: 0.0032
val Loss: 6.0458 Acc: 0.0050

Epoch 2/49
----------
train Loss: 6.0382 Acc: 0.0071
val Loss: 6.0377 Acc: 0.0025

Epoch 3/49
----------
train Loss: 6.0153 Acc: 0.0079
val Loss: 6.0257 Acc: 0.0037

Epoch 4/49
----------
train Loss: 5.9900 Acc: 0.0102
val Loss: 5.9995 Acc: 0.0063

Epoch 5/49
----------
train Loss: 5.9486 Acc: 0.0055
val Loss: 5.9411 Acc: 0.0063

Epoch 6/49
----------
train Loss: 5.8513 Acc: 0.0055
val Loss: 5.7823 Acc: 0.0138

Epoch 7/49
----------
train Loss: 5.7073 Acc: 0.0102
val Loss: 5.6488 Acc: 0.0150

Epoch 8/49
----------
train Loss: 5.5627 Acc: 0.0142
val Loss: 5.4450 Acc: 0.0375

Epoch 9/49
----------
train Loss: 5.3678 Acc: 0.0418
val Loss: 5.3812 Acc: 0.0475

Epoch 10/49
----------
train Loss: 5.0593 Acc: 0.0662
val Loss: 5.1457 Acc: 0.0600

Epoch 11/49
----------
train Loss: 4.8430 Acc: 0.0835
val Loss: 5.0587 Acc: 0.0663

Ep

In [259]:
torch.save(model, 'pretrained/imad_vgg_masked.pt')